In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append("src")

from Models.splinedist import *
import pytorch_lightning as pl
import torch
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor, RichProgressBar, RichModelSummary
from Datasets.DSB18 import * 

In [2]:

logger = pl_loggers.TensorBoardLogger(
    save_dir=os.path.join(os.getcwd(), "pl_logs"),
    name="spline_dist",
    log_graph=True,
    default_hp_metric=True)

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="Models/weights",
    filename="splineDist-{epoch:02d}-{val_loss:.2f}",
    save_top_k=5,
    mode="min",
)

earlystoping_callback = EarlyStopping("val_loss")

lr_logger = LearningRateMonitor(logging_interval="step")
summary = RichModelSummary(max_depth=-1)
progressbar = RichProgressBar()

callbacks = [progressbar, checkpoint_callback, lr_logger]


In [3]:
model = SplineDist()

C:\Users\mbenimam\anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
# trainer = pl.Trainer(max_epochs=2,
#                      gpus=1,
#                      callbacks=callbacks,
#                      logger=logger)
# trainer.fit(model, Nuclie_datamodule())


In [5]:

# torch.onnx.export(model, torch.rand(1, 3, 256, 256), "src/Models/onnx/splineDist.onnx",  opset_version=11)

In [6]:
torch.version.cuda

'10.2'

In [7]:
# import torch
torch.__version__

'1.10.0'

In [8]:
data = Nuclie_datamodule()
batch_x, batch_y = next(iter(data.val_dataloader()))

In [9]:
model = model.cuda()
(objectProbas, angles, distances, controlPoints) = model(batch_x.to("cuda"))

In [10]:
objectProbas.shape, controlPoints.shape

(torch.Size([4, 1, 256, 256]), torch.Size([4, 8, 2, 256, 256]))

In [11]:
torch.cuda.empty_cache()
(targetObjectProbas, targetOverlapProbas, targetContours) = batch_y
B3M = getBsplineMatrix(700, 3, 8).float().to("cuda")

contours = getContourSamples(controlPoints, B3M)

B3M.shape, controlPoints.shape, contours.shape

(torch.Size([700, 8]),
 torch.Size([4, 8, 2, 256, 256]),
 torch.Size([700, 4, 2, 256, 256]))

In [12]:

contours = contours.permute(1, 0, 2, 3, 4).reshape(4, -1, 700, 2)
contours.shape

torch.Size([4, 65536, 700, 2])

In [13]:
targetObjectProbas = targetObjectProbas.to("cuda")
lossObjectProba = F.binary_cross_entropy(objectProbas, targetObjectProbas)


In [14]:
contours.shape, objectProbas.reshape(4, -1).shape

(torch.Size([4, 65536, 700, 2]), torch.Size([4, 65536]))

In [15]:

# loss 2
# non maximum suppression
postProcessed = nonMaximumSuppresionBatch(objectProbas.reshape(4, -1).detach().cpu().numpy(),
                                          contours.detach().cpu().numpy(),
                                          0.5,
                                          0.1)
# find the associated instances from postProcessed with the ground truth


In [ ]:


TP, FP, FN = matchInstances(postProcessed, targetContours)
torch.stack(tuple(computeDistanceBetweenInstance(postProcessed[id1], targetContours[id2]) 
                        for id1, id2 in TP)).sum()
precision = len(TP) / (len(TP) + len(FP) + 1)
recall = len(TP) / (len(TP) + len(FN)+1)
# compute the sum of the distances with predicted instances and the ground truth
lossDistance = 0
# loss 3 sum of the two losses
finalLoss = lossObjectProba + self.lambda1*lossDistance

